In [1]:
from db_queries import get_population, get_ids
from db_queries import get_location_metadata as get_locs

In [2]:
import pandas as pd, numpy as np

# LSFF: choose population coverage data by hand for tier 3 countries

## vehicles: Wheat flour, maize flour, oil

## countries: Kenya, Burkina Faso, Myanmar, Vietnam, Nepal

In [3]:
nutrients = ['iron','zinc','folic acid','vitamin a']

In [4]:
data_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/extraction_sheet_lsff_03_16_2021.csv'
assm_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/extraction_sheet_lsff_assumed_coverage_03_11_2021.csv'

df = pd.read_csv(data_path)

In [5]:
df.loc[df.nutrient=="folic acid, folate, b9",'nutrient']= 'folic acid'

In [6]:
assum = pd.read_csv(assm_path)

In [7]:
assum.location_name.unique()

array(['Ethiopia', 'Myanmar', 'India'], dtype=object)

In [67]:
assum.loc[assum.nutrient=="folic acid, folate, b9",'nutrient']= 'folic acid'

In [8]:
df['estimation_status'] = 'na'

In [9]:
df['data_choice_notes'] = ""

In [10]:
mult_estimates_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_data_estimated_03_11_2021.csv'

mult_estimates = pd.read_csv(mult_estimates_path)

In [11]:
mult_estimates[mult_estimates.location_name=="Pakistan"]

,location_name,vehicle,nutrient,standard,A,B,B_estimate,C,C_estimate
255,Pakistan,wheat flour,iron,Mandatory in Punjab only,97.333333,48.666667,0.0,5.333333,0.0
256,Pakistan,wheat flour,folic acid,Mandatory in Punjab only,97.333333,48.666667,0.0,1.946667,1.0
257,Pakistan,wheat flour,zinc,Unknown,97.333333,48.666667,0.0,1.946667,1.0
258,Pakistan,wheat flour,vitamin b12,Unknown,97.333333,48.666667,0.0,1.946667,1.0
259,Pakistan,wheat flour,vitamin b1,Unknown,97.333333,48.666667,0.0,0.000000,1.0
260,Pakistan,oil,vitamin a,Mandatory,100.000000,95.500000,0.0,30.000000,0.0
261,Pakistan,oil,vitamin d,Mandatory,100.000000,95.500000,0.0,NaN,NaN
348,Pakistan,wheat flour,vitamin a,NaN,97.333333,48.666667,0.0,0.000000,1.0


In [12]:
#reformat
mult_estimates.loc[(mult_estimates.B_estimate!=1.0),'B'] = np.nan
mult_estimates.loc[(mult_estimates.C_estimate!=1.0),'C'] = np.nan

mult_estimates = pd.melt(mult_estimates,
                         id_vars = ['location_name','vehicle','nutrient','standard'],
                         value_vars = ['B','C'], var_name = 'value_description', value_name = 'value_mean').dropna()

mult_estimates.loc[(mult_estimates.value_description=="B"),'nutrient'] = 'NA'

mult_estimates = mult_estimates.drop_duplicates()

mult_estimates.value_description = mult_estimates.value_description.map({
    'B':'percent of population eating industrially produced vehicle',
    'C':'percent of population eating fortified vehicle'
})

In [13]:
mult_estimates['estimation_status'] = 'multiplicative'

In [14]:
## load legal combos
import pickle
data_prep_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/'

with open(data_prep_dir + 'lsff_vehicle_nutrient_pairs.pickle', 'rb') as handle:
    vehicle_nutrient_map = pickle.load(handle)
    
with open(data_prep_dir + 'lsff_country_vehicle_pairs.pickle', 'rb') as handle:
    country_vehicle_map = pickle.load(handle)
    
with open(data_prep_dir + 'lsff_vehicle_country_pairs.pickle', 'rb') as handle:
    vehicle_country_map = pickle.load(handle)

In [15]:
location_names = ['Kenya','Burkina Faso','Myanmar','Vietnam','Nepal']
vehicles = ['maize flour','wheat flour','oil']
nutrients = ['folic acid','iron','zinc','vitamin a']

In [16]:
# these are the vehicles per country we need
target_high_level = pd.DataFrame([(loc,v) for loc in location_names for v in country_vehicle_map[loc]],
            columns=['location_name','vehicle']).sort_values(['location_name','vehicle'])

target_high_level = target_high_level[target_high_level.vehicle.isin(vehicles)].set_index(['location_name','vehicle'])

target_high_level

Empty DataFrame
Columns: []
Index: [(Burkina Faso, maize flour), (Burkina Faso, oil), (Burkina Faso, wheat flour), (Kenya, maize flour), (Kenya, oil), (Kenya, wheat flour), (Myanmar, oil), (Myanmar, wheat flour), (Nepal, oil), (Nepal, wheat flour), (Vietnam, oil), (Vietnam, wheat flour)]

In [17]:
target_a = pd.DataFrame([(loc,vehicle,nutrient,'percent of population eating fortified vehicle') for loc in location_names
                       for vehicle in country_vehicle_map[loc]
                      for nutrient in vehicle_nutrient_map[vehicle]],
            columns=['location_name','vehicle','nutrient','value_description']).sort_values(['location_name','vehicle','nutrient'])
target_a = target_a[(target_a.nutrient.isin(nutrients))]

target_b = pd.DataFrame([(loc,vehicle,'na',val) for loc in location_names
                       for vehicle in country_vehicle_map[loc]
                      for val in ['percent of population eating industrially produced vehicle',
       'percent of population eating vehicle']],
            columns=['location_name','vehicle','nutrient','value_description'])

sortvars = ['location_name','vehicle','value_description','nutrient']
target = target_a.append(target_b)
target = target[(target.vehicle.isin(vehicles)) & (target.nutrient.isin(nutrients))].sort_values(sortvars).set_index(sortvars)

In [18]:
target

Empty DataFrame
Columns: []
Index: [(Burkina Faso, maize flour, percent of population eating fortified vehicle, folic acid), (Burkina Faso, maize flour, percent of population eating fortified vehicle, iron), (Burkina Faso, maize flour, percent of population eating fortified vehicle, vitamin a), (Burkina Faso, maize flour, percent of population eating fortified vehicle, zinc), (Burkina Faso, oil, percent of population eating fortified vehicle, vitamin a), (Burkina Faso, wheat flour, percent of population eating fortified vehicle, folic acid), (Burkina Faso, wheat flour, percent of population eating fortified vehicle, iron), (Burkina Faso, wheat flour, percent of population eating fortified vehicle, vitamin a), (Burkina Faso, wheat flour, percent of population eating fortified vehicle, zinc), (Kenya, maize flour, percent of population eating fortified vehicle, folic acid), (Kenya, maize flour, percent of population eating fortified vehicle, iron), (Kenya, maize flour, percent of population eating fortified vehicle, vitamin a), (Kenya, maize flour, percent of population eating fortified vehicle, zinc), (Kenya, oil, percent of population eating fortified vehicle, vitamin a), (Kenya, wheat flour, percent of population eating fortified vehicle, folic acid), (Kenya, wheat flour, percent of population eating fortified vehicle, iron), (Kenya, wheat flour, percent of population eating fortified vehicle, vitamin a), (Kenya, wheat flour, percent of population eating fortified vehicle, zinc), (Myanmar, oil, percent of population eating fortified vehicle, vitamin a), (Myanmar, wheat flour, percent of population eating fortified vehicle, folic acid), (Myanmar, wheat flour, percent of population eating fortified vehicle, iron), (Myanmar, wheat flour, percent of population eating fortified vehicle, vitamin a), (Myanmar, wheat flour, percent of population eating fortified vehicle, zinc), (Nepal, oil, percent of population eating fortified vehicle, vitamin a), (Nepal, wheat flour, percent of population eating fortified vehicle, folic acid), (Nepal, wheat flour, percent of population eating fortified vehicle, iron), (Nepal, wheat flour, percent of population eating fortified vehicle, vitamin a), (Nepal, wheat flour, percent of population eating fortified vehicle, zinc), (Vietnam, oil, percent of population eating fortified vehicle, vitamin a), (Vietnam, wheat flour, percent of population eating fortified vehicle, folic acid), (Vietnam, wheat flour, percent of population eating fortified vehicle, iron), (Vietnam, wheat flour, percent of population eating fortified vehicle, vitamin a), (Vietnam, wheat flour, percent of population eating fortified vehicle, zinc)]

In [19]:
target = target.reset_index()
rcols = target.columns.tolist()

In [20]:
check_cols = ['location_id','location_name','urbanicity','subnational_name','vehicle','value_description','nutrient','value_mean','value_025_percentile',
       'value_975_percentile','sub_population','source_year','notes','source_citation','source_link','data_choice_notes']

def filter_data(country, vehicle, val):    
    output = df.loc[(df.location_name==country)
           & (df.vehicle==vehicle)
           & (df.value_description==val)
           & (df.value_mean.notna()),check_cols]
    
    return output

In [21]:
def check_one_country(country):
    vehicles = ['oil', 'wheat flour', 'salt', 'maize flour', 'rice', 'bouillon']
    values_gold = ['percent of population eating fortified vehicle',
               'percent of population eating industrially produced vehicle',
               'percent of population eating vehicle']
    return pd.concat([filter_data(country, vehicle, val) for vehicle in vehicles for val in values_gold])

In [22]:
usecols = ['location_id','location_name','subnational_name','vehicle','value_description','nutrient','value_mean', 'value_025_percentile',
       'value_975_percentile']
subset_data = {}

In [23]:
for i in location_names:
    subset_data[i] = pd.DataFrame()

## Kenya

In [28]:
kenya = check_one_country("Kenya")

kenya.value_description.unique()

In [30]:
mult_estimates[(mult_estimates.location_name=="Kenya")]

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status
511,Kenya,wheat flour,iron,Mandatory,percent of population eating fortified vehicle,33.835750,multiplicative
512,Kenya,wheat flour,zinc,Mandatory,percent of population eating fortified vehicle,34.703333,multiplicative
513,Kenya,wheat flour,folic acid,Mandatory,percent of population eating fortified vehicle,34.703333,multiplicative
514,Kenya,wheat flour,vitamin b12,Mandatory,percent of population eating fortified vehicle,34.703333,multiplicative
515,Kenya,wheat flour,vitamin b1,Mandatory,percent of population eating fortified vehicle,34.703333,multiplicative
516,Kenya,wheat flour,vitamin a,Mandatory,percent of population eating fortified vehicle,34.703333,multiplicative
518,Kenya,oil,vitamin d,Unknown,percent of population eating fortified vehicle,0.000000,multiplicative
519,Kenya,maize flour,iron,Mandatory,percent of population eating fortified vehicle,0.000000,multiplicative
520,Kenya,maize flour,zinc,Mandatory,percent of population eating fortified vehicle,0.000000,multiplicative
521,Kenya,maize flour,folic acid,Mandatory,percent of population eating fortified vehicle,0.000000,multiplicative


In [31]:
kenya.groupby(['vehicle','value_description']).mean()

location_id
vehicle     value_description                                              
maize flour percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
oil         percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
wheat flour percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN

In [32]:
kenya.loc[(kenya.vehicle=="maize flour") & (kenya.value_description=="percent of population eating industrially produced vehicle"),
         "data_choice_notes"] = "Only one source."

subset_data["Kenya"] = subset_data['Kenya'].append(kenya[(kenya.vehicle=="maize flour") & (kenya.value_description=="percent of population eating industrially produced vehicle")])

kenya[(kenya.vehicle=="maize flour") & (kenya.value_description=="percent of population eating industrially produced vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
432,NaN,Kenya,mixed/both,na,maize flour,percent of population eating industrially prod...,na,36.67,NaN,NaN,total population,2005,nationally representative survey: proportion o...,"Central Bureau of Statistics (Kenya), UK Depar...",NID 7375,Only one source.


In [39]:
kenya.loc[(kenya.vehicle=="maize flour") 
      & (kenya.value_description=="percent of population eating vehicle")
      & (kenya.source_link=="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5066654/"),
     "data_choice_notes"] = "Excluded central bureau of stats total pop number (85%) from 2005 in lieu of Ferguson rural u5 number from 2015. Note 72% of kenyan population is rural according to the world bank: https://data.worldbank.org/indicator/SP.RUR.TOTL.ZS?locations=KE"

subset_data["Kenya"] = subset_data["Kenya"].append(
kenya.loc[(kenya.vehicle=="maize flour") 
      & (kenya.value_description=="percent of population eating vehicle")
      & (kenya.source_link=="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5066654/")]
)


kenya[(kenya.vehicle=="maize flour") 
      & (kenya.value_description=="percent of population eating vehicle")]


,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
447,NaN,Kenya,mixed/both,na,maize flour,percent of population eating vehicle,na,85.27,NaN,NaN,total population,2005,nationally representative survey: proportion o...,"Central Bureau of Statistics (Kenya), UK Depar...",NID 7375,
503,NaN,Kenya,rural,Kitui,maize flour,percent of population eating vehicle,na,57,NaN,NaN,6-8 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
504,NaN,Kenya,rural,Kitui,maize flour,percent of population eating vehicle,na,63,NaN,NaN,9-11 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
505,NaN,Kenya,rural,Kitui,maize flour,percent of population eating vehicle,na,81,NaN,NaN,12-23 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
506,NaN,Kenya,rural,Kitui,maize flour,percent of population eating vehicle,na,16,NaN,NaN,6-8 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
507,NaN,Kenya,rural,Kitui,maize flour,percent of population eating vehicle,na,38,NaN,NaN,9-11 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
508,NaN,Kenya,rural,Kitui,maize flour,percent of population eating vehicle,na,17,NaN,NaN,12-23 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
509,NaN,Kenya,rural,Vihiga,maize flour,percent of population eating vehicle,na,89,NaN,NaN,6-8 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
510,NaN,Kenya,rural,Vihiga,maize flour,percent of population eating vehicle,na,91,NaN,NaN,9-11 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
511,NaN,Kenya,rural,Vihiga,maize flour,percent of population eating vehicle,na,95,NaN,NaN,12-23 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...


In [ ]:
# kenya.loc[(kenya.vehicle=="maize flour") & (kenya.value_description=="percent of population eating industrially produced vehicle"),
#          "data_choice_notes"] = "Only one source."

# subset_data["Kenya"] = subset_data['Kenya'].append(kenya[(kenya.vehicle=="maize flour") & (kenya.value_description=="percent of population eating industrially produced vehicle")])

# kenya[(kenya.vehicle=="maize flour") & (kenya.value_description=="percent of population eating industrially produced vehicle")]

In [42]:
kenya.loc[(kenya.vehicle=="oil") 
      & (kenya.value_description=="percent of population eating industrially produced vehicle")
          ,"data_choice_notes"] = "Only one source."

subset_data['Kenya'] = subset_data['Kenya'].append(
kenya[(kenya.vehicle=="oil") 
      & (kenya.value_description=="percent of population eating industrially produced vehicle")]
)

kenya[(kenya.vehicle=="oil") 
      & (kenya.value_description=="percent of population eating industrially produced vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
445,NaN,Kenya,mixed/both,na,oil,percent of population eating industrially prod...,na,9.39,NaN,NaN,total population,2005,nationally representative survey: proportion o...,"Central Bureau of Statistics (Kenya), UK Depar...",NID 7375,Only one source.


In [44]:
kenya.loc[(kenya.vehicle=="oil") 
      & (kenya.value_description=="percent of population eating vehicle")
      & (kenya.source_link=="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5066654/"),
     "data_choice_notes"] = "Excluded central bureau of stats total pop number (16%) from 2005 in lieu of Ferguson rural u5 number from 2015. Note 72% of kenyan population is rural according to the world bank: https://data.worldbank.org/indicator/SP.RUR.TOTL.ZS?locations=KE"

subset_data["Kenya"] = subset_data["Kenya"].append(
kenya.loc[(kenya.vehicle=="oil") 
      & (kenya.value_description=="percent of population eating vehicle")
      & (kenya.source_link=="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5066654/")]
)


kenya[(kenya.vehicle=="oil") 
      & (kenya.value_description=="percent of population eating vehicle")]


,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
449,NaN,Kenya,mixed/both,na,oil,percent of population eating vehicle,na,16.17,NaN,NaN,total population,2005,nationally representative survey: proportion o...,"Central Bureau of Statistics (Kenya), UK Depar...",NID 7375,
514,NaN,Kenya,rural,Kitui,oil,percent of population eating vehicle,na,6,NaN,NaN,6-8 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
515,NaN,Kenya,rural,Kitui,oil,percent of population eating vehicle,na,16,NaN,NaN,9-11 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
516,NaN,Kenya,rural,Kitui,oil,percent of population eating vehicle,na,7,NaN,NaN,12-23 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
517,NaN,Kenya,rural,Kitui,oil,percent of population eating vehicle,na,12,NaN,NaN,6-8 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
518,NaN,Kenya,rural,Kitui,oil,percent of population eating vehicle,na,38,NaN,NaN,9-11 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
519,NaN,Kenya,rural,Kitui,oil,percent of population eating vehicle,na,45,NaN,NaN,12-23 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
520,NaN,Kenya,rural,Vihiga,oil,percent of population eating vehicle,na,60,NaN,NaN,6-8 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
521,NaN,Kenya,rural,Vihiga,oil,percent of population eating vehicle,na,70,NaN,NaN,9-11 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...
522,NaN,Kenya,rural,Vihiga,oil,percent of population eating vehicle,na,72,NaN,NaN,12-23 months,2015,population = children who were breastfeeding. ...,"Ferguson, Elaine et al. “Zinc, iron and calciu...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Excluded central bureau of stats total pop num...


In [46]:
kenya.loc[(kenya.vehicle=="wheat flour") & (kenya.value_description=="percent of population eating industrially produced vehicle"),
         "data_choice_notes"] = "Only one source."

subset_data["Kenya"] = subset_data['Kenya'].append(kenya[(kenya.vehicle=="wheat flour") & (kenya.value_description=="percent of population eating industrially produced vehicle")])

kenya[(kenya.vehicle=="wheat flour") & (kenya.value_description=="percent of population eating industrially produced vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
444,NaN,Kenya,mixed/both,na,wheat flour,percent of population eating industrially prod...,na,30.95,NaN,NaN,total population,2005,nationally representative survey: proportion o...,"Central Bureau of Statistics (Kenya), UK Depar...",NID 7375,Only one source.


In [48]:
kenya.loc[(kenya.vehicle=="wheat flour") & (kenya.value_description=="percent of population eating vehicle"),
         "data_choice_notes"] = "Only one source."

subset_data["Kenya"] = subset_data['Kenya'].append(kenya[(kenya.vehicle=="wheat flour") & (kenya.value_description=="percent of population eating vehicle")])

kenya[(kenya.vehicle=="wheat flour") & (kenya.value_description=="percent of population eating vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
448,NaN,Kenya,mixed/both,na,wheat flour,percent of population eating vehicle,na,36.58,NaN,NaN,total population,2005,nationally representative survey: proportion o...,"Central Bureau of Statistics (Kenya), UK Depar...",NID 7375,Only one source.


In [52]:
#need pct eating fortified for oil, wheat, maize
subset_data["Kenya"] = subset_data['Kenya'].append(
    mult_estimates[(mult_estimates.location_name=="Kenya") & (mult_estimates.nutrient.isin(nutrients))]
)

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v107/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


## Myanmar

In [55]:
myanmar = check_one_country("Myanmar")

myanmar.groupby(['vehicle','value_description']).mean()

,,location_id
vehicle,value_description,
oil,percent of population eating industrially produced vehicle,NaN


In [76]:
target[target.location_name=="Myanmar"]

,location_name,vehicle,value_description,nutrient
18,Myanmar,oil,percent of population eating fortified vehicle,vitamin a
19,Myanmar,wheat flour,percent of population eating fortified vehicle,folic acid
20,Myanmar,wheat flour,percent of population eating fortified vehicle,iron
21,Myanmar,wheat flour,percent of population eating fortified vehicle,vitamin a
22,Myanmar,wheat flour,percent of population eating fortified vehicle,zinc


In [58]:
myanmar

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
595,NaN,Myanmar,NaN,na,oil,percent of population eating industrially prod...,na,10,0,20,total population,NaN,NaN,NaN,Flander’s investment trade report,


In [60]:
myanmar.loc[(myanmar.vehicle=="oil")
            & (myanmar.value_description=="percent of population eating industrially produced vehicle")
            ,"data_choice_notes"] = "Only one source."
subset_data['Myanmar'] = subset_data['Myanmar'].append(myanmar.loc[(myanmar.vehicle=="oil")
            & (myanmar.value_description=="percent of population eating industrially produced vehicle")])

myanmar.loc[(myanmar.vehicle=="oil")
            & (myanmar.value_description=="percent of population eating industrially produced vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
595,NaN,Myanmar,NaN,na,oil,percent of population eating industrially prod...,na,10,0,20,total population,NaN,NaN,NaN,Flander’s investment trade report,Only one source.


In [69]:
# we'll discard this in lieu of a regression estimate
assum[(assum.location_name=="Myanmar") 
      & (assum.value_description=="percent of population eating vehicle") 
      & (assum.vehicle=="oil")]

,location_id,location_name,vehicle,nutrient,value_description,value_mean,value_025_percentile,value_975_percentile,sub_population,notes,Unnamed: 10
4,15,Myanmar,oil,na,percent of population eating vehicle,50,0,100,total population,"uninformative prior – 100% eat, and a lot! (ac...",NaN


In [74]:
assum.loc[(assum.location_name=="Myanmar") 
      & (assum.value_description=="percent of population eating fortified vehicle") 
      & (assum.vehicle=="oil")
     & (assum.nutrient.isin(nutrients)),
         "data_choice_notes"] = "Unable to find data; assuming 0; justification: lack of reports on fortification activities."

subset_data['Myanmar'] = subset_data['Myanmar'].append(
assum[(assum.location_name=="Myanmar") 
      & (assum.value_description=="percent of population eating fortified vehicle") 
      & (assum.vehicle=="oil")
     & (assum.nutrient.isin(nutrients))]
)

assum[(assum.location_name=="Myanmar") 
      & (assum.value_description=="percent of population eating fortified vehicle") 
      & (assum.vehicle=="oil")
     & (assum.nutrient.isin(nutrients))]

,location_id,location_name,vehicle,nutrient,value_description,value_mean,value_025_percentile,value_975_percentile,sub_population,notes,Unnamed: 10,data_choice_notes
3,15,Myanmar,oil,vitamin a,percent of population eating fortified vehicle,0,0,5,total population,Justification - lack of reports on fortificati...,NaN,Unable to find data; assuming 0; justification...


In [78]:
#we'll discard this for a regression estimate
assum[(assum.location_name=="Myanmar") 
      & (assum.value_description=="percent of population eating industrially produced vehicle") 
      & (assum.vehicle=="wheat flour")]

,location_id,location_name,vehicle,nutrient,value_description,value_mean,value_025_percentile,value_975_percentile,sub_population,notes,Unnamed: 10,data_choice_notes
9,15,Myanmar,wheat flour,na,percent of population eating industrially prod...,50,0,100,total population,uninformative prior,NaN,NaN


In [80]:
# ... take this?
assum.loc[(assum.location_name=="Myanmar") 
      & (assum.value_description=="percent of population eating vehicle") 
      & (assum.vehicle=="wheat flour"),
         "data_choice_notes"] = "Made assumption due to lack of data. Justification: FAO food balance sheets and general indication that rice is much more common"

subset_data['Myanmar'] = subset_data['Myanmar'].append(
assum[(assum.location_name=="Myanmar") 
      & (assum.value_description=="percent of population eating vehicle") 
      & (assum.vehicle=="wheat flour")]
)

assum[(assum.location_name=="Myanmar") 
      & (assum.value_description=="percent of population eating vehicle") 
      & (assum.vehicle=="wheat flour")]

,location_id,location_name,vehicle,nutrient,value_description,value_mean,value_025_percentile,value_975_percentile,sub_population,notes,Unnamed: 10,data_choice_notes
10,15,Myanmar,wheat flour,na,percent of population eating vehicle,5,0,10,total population,FAO food balance sheets and general indication...,NaN,Made assumption due to lack of data. Justifica...


In [84]:
assum.loc[(assum.location_name=="Myanmar") 
      & (assum.value_description=="percent of population eating fortified vehicle") 
      & (assum.vehicle=="wheat flour")
     & (assum.nutrient.isin(nutrients)),
         "data_choice_notes"] = "Unable to find data; assuming 0; justification: lack of reports on fortification activities."

subset_data['Myanmar'] = subset_data['Myanmar'].append(
assum[(assum.location_name=="Myanmar") 
      & (assum.value_description=="percent of population eating fortified vehicle") 
      & (assum.vehicle=="wheat flour")
     & (assum.nutrient.isin(nutrients))]
)

assum[(assum.location_name=="Myanmar") 
      & (assum.value_description=="percent of population eating fortified vehicle") 
      & (assum.vehicle=="wheat flour")
     & (assum.nutrient.isin(nutrients))]

,location_id,location_name,vehicle,nutrient,value_description,value_mean,value_025_percentile,value_975_percentile,sub_population,notes,Unnamed: 10,data_choice_notes
6,15,Myanmar,wheat flour,folic acid,percent of population eating fortified vehicle,0,0,5,total population,Justification - lack of reports on fortificati...,NaN,Unable to find data; assuming 0; justification...
7,15,Myanmar,wheat flour,iron,percent of population eating fortified vehicle,0,0,5,total population,Justification - lack of reports on fortificati...,NaN,Unable to find data; assuming 0; justification...
8,15,Myanmar,wheat flour,zinc,percent of population eating fortified vehicle,0,0,5,total population,Justification - lack of reports on fortificati...,NaN,Unable to find data; assuming 0; justification...


In [86]:
mult_estimates[(mult_estimates.location_name=="Myanmar")]

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status
554,Myanmar,wheat flour,vitamin b12,Unknown,percent of population eating fortified vehicle,9.166667,multiplicative
555,Myanmar,wheat flour,vitamin b1,Unknown,percent of population eating fortified vehicle,9.166667,multiplicative


In [89]:
outputs_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/'

pct_eating = pd.read_csv(outputs_dir + 'pct_eating_regression_estimates_3_15_2021.csv')
pct_ind_prod = pd.read_csv(outputs_dir + 'pct_eating_ind_prod_regression_estimates_3_15_2021.csv')
pct_fortified = pd.read_csv(outputs_dir + 'pct_eating_fortified_regression_estimates_3_15_2021.csv')

In [91]:
pct_eating[pct_eating.location_name=="Myanmar"]

,location_name,vehicle,percent_of_population_eating_vehicle
97,Myanmar,oil,50.0
98,Myanmar,oil,50.0


In [92]:
pct_ind_prod[pct_ind_prod.location_name=="Myanmar"]

,location_name,vehicle,percent_of_population_eating_industrially_produced_vehicle
15,Myanmar,oil,10.0


In [93]:
pct_fortified[pct_fortified.location_name=="Myanmar"]

,location_name,vehicle,percent_of_population_eating_fortified_vehicle
15,Myanmar,oil,0.0


In [94]:
# COME BACK TO MYANMAR THIS IS A MESS